# Modèle V1 de voiture autonome sur DonkeyCarSimulator

ATTENTION : 
- Modifier JPEG/JPG ou PNG
- Modifier Color (gray or 3 colors)

Verifier `#XXX`

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from time import time
from tqdm import tqdm
import json
from PIL import Image
import base64
from io import BytesIO
import inspect

In [ ]:
TIME=str(time())
MODEL_NAME = "DCDeepModelV2.1-renault-"+TIME

STORAGE_ROOT_DIR = "drive/My Drive/ColabStorage/DonkeyCar/Simulator"
TRAINER_PROJECT_PATH = "Trainer"

DATASET_NAME = "corentin_renault_8600_cam_conf"
DATASET_PATH =DATASET_NAME
DATASET_LABEL_FILENAME = "label.csv"
DATASET_LABEL_PATH = os.path.join(DATASET_PATH, DATASET_LABEL_FILENAME)

TENSORLOG_PATH = os.path.join(STORAGE_ROOT_DIR, TRAINER_PROJECT_PATH, "Log", MODEL_NAME)
os.mkdir(TENSORLOG_PATH)

BATCH_SIZE = 128

SAVE_PATH = os.path.join(STORAGE_ROOT_DIR, "Model", MODEL_NAME)
os.mkdir(SAVE_PATH)

IMAGE_SHAPE = (120,160, 3)
print("Ok.")

Ok.


# 0. Extraire la dataset

## A) zip en dataset

In [ ]:
!rm -Rf "/home/nigiva/git/dkarstream/Samples/3_controller_generated_track_0_48000"
!unzip -q "/home/nigiva/git/dkarstream/Samples/3_controller_generated_track_0_48000.zip" -d "/home/nigiva/git/dkarstream/Samples/"
!ls "/home/nigiva/git/dkarstream/Samples"

3_controller_generated_track_0_48000  3_controller_generated_track_0_48000.zip


## B) json (.eslr) en dataset
On convertit ligne par ligne les données envoyées par le serveur en :
- une image
- une ligne dans le csv label.csv

In [ ]:
!rm -Rf "corentin_renault_8600_cam_conf"

In [ ]:
!cp "drive/My Drive/ColabStorage/DonkeyCar/Simulator/Dataset/corentin_renault_8600_cam_conf.eslr" "corentin_renault_8600_cam_conf.eslr"

In [ ]:
DATASET_JSON_FILE = DATASET_PATH + ".eslr"

IMAGE_PATH = "images"
DATASET_IMAGE_PATH = os.path.join(DATASET_PATH, IMAGE_PATH)

if not os.path.exists(DATASET_PATH):
    os.mkdir(DATASET_PATH)
if not os.path.exists(DATASET_IMAGE_PATH):
    os.mkdir(DATASET_IMAGE_PATH)

In [ ]:
DATA_LABEL_FILE = open(DATASET_LABEL_PATH, "w")
DATA_LABEL_FILE.write("path,angle,throttle\n")
with open(DATASET_JSON_FILE, "r") as dataset_file:
    for i, line in enumerate(tqdm(dataset_file)):
        data_line = json.loads(line)
        if (data_line["msg_type"] == "telemetry"):
            image_relative_path = os.path.join(IMAGE_PATH, str(i)+".jpeg") #XXX
            image_absolute_path = os.path.join(DATASET_PATH, image_relative_path)
            Image.open(BytesIO(base64.b64decode(data_line["image"]))).save(image_absolute_path)
            # Toutes les données à ajouter dans la dataset
            data2write = [image_relative_path, str(data_line["steering_angle"]), str(data_line["throttle"])]
            DATA_LABEL_FILE.write(",".join(data2write) + "\n")
DATA_LABEL_FILE.close()
print("ok.")

8627it [00:12, 704.11it/s]

ok.


# 1. Préparer la dataset

In [ ]:
# Obetnir les valeurs du fichier contenant les labels
raw_data = pd.read_csv(DATASET_LABEL_PATH)

# change la chemin de fichier
def change_path(path):
  return os.path.join(DATASET_PATH, path)
raw_data['path'] = raw_data['path'].map(change_path)
print(raw_data)

                                                 path  angle  throttle
0        corentin_renault_8600_cam_conf/images/0.jpeg    0.0       0.6
1        corentin_renault_8600_cam_conf/images/1.jpeg    0.0       0.6
2        corentin_renault_8600_cam_conf/images/2.jpeg    0.0       0.6
3        corentin_renault_8600_cam_conf/images/3.jpeg    0.0       0.6
4        corentin_renault_8600_cam_conf/images/4.jpeg    0.0       0.6
...                                               ...    ...       ...
8622  corentin_renault_8600_cam_conf/images/8622.jpeg    0.0       0.6
8623  corentin_renault_8600_cam_conf/images/8623.jpeg    0.0       0.6
8624  corentin_renault_8600_cam_conf/images/8624.jpeg    0.0       0.6
8625  corentin_renault_8600_cam_conf/images/8625.jpeg    0.0       0.6
8626  corentin_renault_8600_cam_conf/images/8626.jpeg    0.0       0.6

[8627 rows x 3 columns]


### Split en 3 jeux : Train, Test et Validation

In [ ]:
train_and_test_set, validation_set = train_test_split(raw_data,
                                             test_size = 0.15,
                                             shuffle = True)
train_set, test_set = train_test_split(train_and_test_set,
                                             test_size = 0.20,
                                             shuffle = True)
NBR_ROW_TRAIN_SET = train_set.shape[0]
NBR_ROW_TEST_SET = test_set.shape[0]
NBR_ROW_VALIDATION_SET = validation_set.shape[0]
print(train_set)
print(test_set)
print(validation_set)

                                                 path  angle  throttle
6819  corentin_renault_8600_cam_conf/images/6819.jpeg   0.00       0.6
4208  corentin_renault_8600_cam_conf/images/4208.jpeg   0.64       0.2
7755  corentin_renault_8600_cam_conf/images/7755.jpeg  -0.64       0.2
6621  corentin_renault_8600_cam_conf/images/6621.jpeg   0.64       0.2
1587  corentin_renault_8600_cam_conf/images/1587.jpeg   0.64       0.2
...                                               ...    ...       ...
7459  corentin_renault_8600_cam_conf/images/7459.jpeg   0.00       0.6
8330  corentin_renault_8600_cam_conf/images/8330.jpeg   0.64       0.2
259    corentin_renault_8600_cam_conf/images/259.jpeg  -0.64       0.2
6556  corentin_renault_8600_cam_conf/images/6556.jpeg   0.00       0.6
8293  corentin_renault_8600_cam_conf/images/8293.jpeg   0.00       0.6

[5865 rows x 3 columns]
                                                 path     angle  throttle
4439  corentin_renault_8600_cam_conf/images/4439.

### Traitements avec TensorData

In [ ]:
# Mettre dans des tensors
#train_tensor = tf.data.Dataset.from_tensor_slices(({"input" : train_set['path']}, {"angle" : train_set['angle'], "throttle" : train_set['throttle']}))
#test_tensor = tf.data.Dataset.from_tensor_slices(({"input" : test_set['path']}, {"angle" : test_set['angle'], "throttle" : test_set['throttle']}))
#validation_tensor = tf.data.Dataset.from_tensor_slices(({"input" : validation_set['path']}, {"angle" : validation_set['angle'], "throttle" : validation_set['throttle']}))
train_tensor = tf.data.Dataset.from_tensor_slices(({"input" : train_set['path']}, {"angle" : train_set['angle']}))
test_tensor = tf.data.Dataset.from_tensor_slices(({"input" : test_set['path']}, {"angle" : test_set['angle']}))
validation_tensor = tf.data.Dataset.from_tensor_slices(({"input" : validation_set['path']}, {"angle" : validation_set['angle']}))

# Definir les fonctions de chargement des images et de mapping
def load_and_preprocess_image(path):
    file_content = tf.io.read_file(path['input'])
    tricolors_img = tf.cast(tf.image.decode_jpeg(file_content, channels=3), dtype=tf.float32) #XXX
    #gray_img = tf.image.rgb_to_grayscale(tricolors_img) #XXX
    normalized_img = (tricolors_img / 127.5) - 1
    normalized_img = tf.reshape(normalized_img, IMAGE_SHAPE)
    print(normalized_img)
    return {"input": normalized_img}

def load_map_function(path, d):
    return load_and_preprocess_image(path), d

# Appliquer le mapping aux tensors
train_tensor_normalized = train_tensor.map(load_map_function, num_parallel_calls=3)
test_tensor_normalized = test_tensor.map(load_map_function, num_parallel_calls=3)
validation_tensor_normalized = validation_tensor.map(load_map_function, num_parallel_calls=3)

Tensor("Reshape:0", shape=(120, 160, 3), dtype=float32)
Tensor("Reshape:0", shape=(120, 160, 3), dtype=float32)
Tensor("Reshape:0", shape=(120, 160, 3), dtype=float32)


In [ ]:
# On mélange les datasets (on fixe un nombre d'exemple tiré au sort, ici 20 000 exemples sur les 30 000)
# On demande un prechargement à l'avance de toujours 3 exemples
train_dataset = train_tensor_normalized.shuffle(6000).batch(BATCH_SIZE).prefetch(2)
test_dataset = test_tensor_normalized.shuffle(1000).batch(BATCH_SIZE).prefetch(2)
validation_dataset = validation_tensor_normalized.shuffle(1000).batch(BATCH_SIZE).prefetch(2)

# 2. Le modèle

In [ ]:
class ModelSaver:
  """
  Save the model into a file ('model.code') as source code
  """
  def __init__(self, activate = True):
    """
    Initialize the saver
    :param activate: capture or not the source code of decorated functions
    """
    self.is_activated = activate
    self.s_init = None
    self.s_call = None
  def init(self, funct):
    """
    Capture the source code of the init function
    ---
    Use as a decoration
    Such as :
    ```
    MODEL_SAVER = ModelSaver(True)
    ...
    @MODEL_SAVER.init
    def __init__(self, name = ""):
      ...
    ```
    """
    def funct_with_params(*args, **kwargs):
      return funct(*args, **kwargs)
    if self.is_activated and self.s_init is None:
      self.s_init = inspect.getsource(funct)
    return funct_with_params

  def call(self, funct):
    """
    Capture the source code of the init function
    ---
    Use as a decoration
    Such as :
    ```
        MODEL_SAVER = ModelSaver(True)
        ...
        @MODEL_SAVER.call
        def call(self):
            ...
    ```
    """
    def funct_with_params(*args, **kwargs):
      return funct(*args, **kwargs)
    if self.is_activated and self.s_call is None:
      self.s_call = inspect.getsource(funct)
    return funct_with_params
  
  def save(self, path):
    """
    Save the source code of the model as a file
    :param path: file path
    """
    if self.s_init is not None and self.s_call is not None:
      with open(path, "w") as s:
        s.write("class DCModel(keras.Model):\n")
        s.write("  MODEL_SAVER = ModelSaver(False)\n")
        s.write(self.s_init)
        s.write(self.s_call)
    else:
      raise Exception("init or call function are not saved")
  @staticmethod
  def load(path):
    """
    Load the Model source code
    :param path: file path
    """
    with open(path, "r") as s:
      exec(s.read())

In [ ]:
LR = 0.001

In [ ]:
MODEL_SAVER = ModelSaver(True)

class DCModel(keras.Model):
  @MODEL_SAVER.init
  def __init__(self, name="DCModel"):
    super(DCModel, self).__init__(name=name)
    #self.input_layer = keras.layers.Input(shape=IMAGE_SHAPE, name='input')
    self.cnn_1 = keras.layers.Conv2D(12, (5, 5), strides=(2, 2), padding="same", kernel_initializer='he_uniform', activation='relu', name='input2')
    self.bn_1 = tf.keras.layers.BatchNormalization()

    self.cnn_2 = keras.layers.Conv2D(16, (5, 5), strides=(2, 2), padding="same", kernel_initializer='he_uniform', activation='relu')
    self.bn_2 = tf.keras.layers.BatchNormalization()

    self.cnn_3 = keras.layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same", kernel_initializer='he_uniform', activation='relu')
    self.bn_3 = tf.keras.layers.BatchNormalization()

    self.cnn_4 = keras.layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same", kernel_initializer='he_uniform', activation='relu')
    self.bn_4 = tf.keras.layers.BatchNormalization()

    self.cnn_5 = keras.layers.Conv2D(48, (3, 3), strides=(2, 2), padding="same", kernel_initializer='he_uniform', activation='relu')
    self.bn_5 = tf.keras.layers.BatchNormalization()

    self.drop_1 = keras.layers.Dropout(0.2)

    self.flat = keras.layers.Flatten()

    self.dense_1 = keras.layers.Dense(100, kernel_initializer='he_uniform', activation='elu')
    self.bn_6 = tf.keras.layers.BatchNormalization()
    self.drop_2 = keras.layers.Dropout(0.1)

    self.dense_2 = keras.layers.Dense(50, kernel_initializer='he_uniform', activation='elu')
    self.bn_7 = tf.keras.layers.BatchNormalization()
    self.drop_3 = keras.layers.Dropout(0.1)

    self.dense_3 = keras.layers.Dense(25, kernel_initializer='he_uniform', activation='elu')
    self.bn_8 = tf.keras.layers.BatchNormalization()
    self.drop_4 = keras.layers.Dropout(0.1)

    self.dense_4 = keras.layers.Dense(25, kernel_initializer='he_uniform', activation='elu')
    self.bn_9 = tf.keras.layers.BatchNormalization()

    self.output_layer_1 = keras.layers.Dense(1, activation='linear', name='angle')
    self.output_layer_2 = keras.layers.Dense(1, activation='linear', name='throttle')
  
  @MODEL_SAVER.call
  @tf.function
  def call(self, inputs, training=False):
    l = self.cnn_1(inputs['input'])
    l = self.bn_1(l)
    
    l = self.cnn_2(l)
    l = self.bn_2(l)

    l = self.cnn_3(l)
    l = self.bn_3(l)

    l = self.cnn_4(l)
    l = self.bn_4(l)

    l = self.cnn_5(l)
    l = self.bn_5(l)

    l = self.drop_1(l)

    l = self.flat(l)

    l = self.dense_1(l)
    l = self.bn_6(l)
    l = self.drop_2(l)

    l = self.dense_2(l)
    l = self.bn_7(l)
    l = self.drop_3(l)

    l = self.dense_3(l)
    l = self.bn_8(l)
    l = self.drop_4(l)

    l = self.dense_4(l)
    l = self.bn_9(l)
    
    return {'angle' : self.output_layer_1(l)}

model = DCModel(name='DonkeyCarModel')

optimizer = keras.optimizers.Adam(learning_rate=LR)
model.compile(optimizer=optimizer,loss=keras.losses.MSE, metrics=["mse"])

## 3. L'entrainement

In [ ]:
#@title Les hyperparamètres de l'entrainement
INITIAL_EPOQUE = 0
NBR_EPOQUES = 20
NBR_EPOQUES_APRES_EARLY_STOPPING = 4

In [ ]:
# Définir mes callback
cb = [
    keras.callbacks.EarlyStopping(patience=NBR_EPOQUES_APRES_EARLY_STOPPING,
                                  restore_best_weights=True),
    keras.callbacks.TensorBoard(log_dir=TENSORLOG_PATH)
    ]

"""
 keras.callbacks.ModelCheckpoint(filepath=SAVE_PATH,
                                    save_weights_only=True,
                                    monitor='val_loss',
                                    mode='min',
                                    save_best_only=True)
"""

# Fit mon modèle
model.fit(train_dataset,
          validation_data=test_dataset,
          epochs=NBR_EPOQUES,
          initial_epoch = INITIAL_EPOQUE,
          callbacks=cb)

Epoch 1/20
46/46 [==============================] - 37s 104ms/step - loss: 1.1159 - mse: 1.1159 - val_loss: 0.0827 - val_mse: 0.0827
Epoch 2/20
46/46 [==============================] - 6s 71ms/step - loss: 0.2018 - mse: 0.2018 - val_loss: 0.0867 - val_mse: 0.0867
Epoch 3/20
46/46 [==============================] - 6s 71ms/step - loss: 0.1052 - mse: 0.1052 - val_loss: 0.0835 - val_mse: 0.0835
Epoch 4/20
46/46 [==============================] - 6s 71ms/step - loss: 0.0733 - mse: 0.0733 - val_loss: 0.0654 - val_mse: 0.0654
Epoch 5/20
46/46 [==============================] - 6s 70ms/step - loss: 0.0564 - mse: 0.0564 - val_loss: 0.0483 - val_mse: 0.0483
Epoch 6/20
46/46 [==============================] - 6s 71ms/step - loss: 0.0497 - mse: 0.0497 - val_loss: 0.0381 - val_mse: 0.0381
Epoch 7/20
46/46 [==============================] - 6s 71ms/step - loss: 0.0400 - mse: 0.0400 - val_loss: 0.0371 - val_mse: 0.0371
Epoch 8/20
46/46 [==============================] - 6s 71ms/step - loss: 0.0348 -

# 4. Sauvegarde

In [ ]:
#model.save(SAVE_PATH, save_format='tf')
model.save_weights(os.path.join(SAVE_PATH, "weights.data"))
MODEL_SAVER.save(os.path.join(SAVE_PATH, "model.code"))

In [ ]:
print(MODEL_NAME)

DCDeepModelV2.1-renault-1615027364.7771797


In [ ]:
model.summary()

Model: "DonkeyCarModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input2 (Conv2D)              multiple                  912       
_________________________________________________________________
batch_normalization (BatchNo multiple                  48        
_________________________________________________________________
conv2d (Conv2D)              multiple                  4816      
_________________________________________________________________
batch_normalization_1 (Batch multiple                  64        
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4640      
_________________________________________________________________
batch_normalization_2 (Batch multiple                  128       
_________________________________________________________________
conv2d_2 (Conv2D)            multiple               

ValueError: ignored

In [ ]:
#Tensorboad
%load_ext tensorboard
%tensorboard --logdir="drive/My Drive/ColabStorage/DonkeyCar/Simulator/Trainer/Log"

In [ ]:
performances = model.evaluate(validation_dataset, batch_size=32)
#print("Loss Angle :", performances[0],"Loss Throttle :", performances[1],"Acc Angle :", performances[2],"Acc Throttle :", performances[3])
print(performances)

11/11 [==============================] - 1s 38ms/step - loss: 0.0070 - mse: 0.0070
[0.007006463129073381, 0.007006463129073381]
